<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/Comparing_fine_tuning_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using BERT models

This example shows how to use BERT models from the HuggingFace library. We will first take a look at the basic usage and how we apply the model, and then see how we can fine-tune the BERT model to classify documents.

To run this example, you need to make sure that the library `transformers` is installed. Please refer to [this page](https://huggingface.co/transformers/installation.html) for installation instructions, but in short you just need to install a package with `pip`. If you use Colab, the library is already installed.

**Note**: If you are using Colab, please check that the runtime type is set to GPU after copying this notebook.

### Using the BERT tokenizer

BERT has a built-in tokenizer. It uses WordPiece tokenization, which means that it will split rare words into pieces.

There are many models included in the `transformers` library, each one identified by a model name string. For a full list, see [here](https://huggingface.co/transformers/pretrained_models.html). We will mainly consider the *BERT* model, originally described in the [paper by Devlin et al. (2019)](https://aclanthology.org/N19-1423.pdf).

Each model in the `transformers` library comes with its own corresponding tokenizer: `BertTokenizer`, `DistilBertTokenizer`, `RobertaTokenizer`, etc. You need to make sure you use the right tokenizer for your model. Fortunately, in the latest version of the library, there is a utility called `AutoTokenizer` that will load the appropriate tokenizer for the model you have selected. Let's see how to use it.

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
import torch
from torch import nn
from torch.utils.data import DataLoader
import pickle
import numpy as np


# Tokenization in Transformer models

Please look at the BERT notebook for more discussions about tokenizers.

In [ ]:
from transformers import AutoTokenizer

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

To exemplify how the tokenizer works, let's see an example. Note that we have added the special dummy tokens BERT uses: `[CLS]` to mark the beginning of the text, `[SEP]` at the end, and some padding.

In [ ]:
tokenizer.tokenize('[CLS] Rolf lives in Gothenburg. [SEP] [PAD] [PAD]')

The following example shows how rare words are split into word pieces by the BERT tokenizer.

In [ ]:
tokenizer.tokenize('Suzanna has broken the teapots.')

We can also encode the tokens as integers. (Each BERT tokenizer comes with a vocabulary that carries out this mapping.) The encoding step automatically adds the `[CLS]` and `[SEP]` dummy tokens at the beginning and end, which is why we get 7 integers for 5 tokens.

In [ ]:
tokenizer(['Rolf lives in Gothenburg.', 'Shorter example'],
         truncation=True, max_length=128,
         return_tensors='pt', padding=True)

Testing the tokenizer:

In [ ]:
tokenizer('tokenization')

{'input_ids': [30001, 1634], 'attention_mask': [1, 1]}

In [ ]:
encoded = tokenizer('Tokenizations and miscalculations.').input_ids
encoded

[30642, 4582, 290, 285, 7860, 3129, 602, 13]

In [ ]:
tokenizer.unk_token_id

50256

In [ ]:
tokenizer.decode(encoded)

'Tokenizations and miscalculations.'

In [ ]:
for i in encoded:
  print(f'{i}\t|{tokenizer.decode(i)}|')

30642	|Token|
4582	|izations|
290	| and|
285	| m|
7860	|iscal|
3129	|cul|
602	|ations|
13	|.|


In [ ]:
tokenizer('Testing PyTorch.', return_tensors='pt')

{'input_ids': tensor([[44154,  9485, 15884,   354,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

# Loading the GPT-2 model from the library

*   List item
*   List item



As we saw in the other notebook, we generally use the 'Auto' classes in Transformers to load models, and provide a model name. In this case, we specity `gpt2` to load the GPT-2 model.

One small detail: in this case, we want to use GPT-2 as a *language model* -- that is, we want to apply it in a left-to-right fashion, in each step computing a distribution over the possible next words. For this type of application, we use `AutoModelForCausalLM` instead of `AutoModel` which is a pure language model. (Terminological note: The term *causal* LM simply means autoregressive LM in this context. It is unrelated to causality-based machine learning models.)

### Trying out a BERT model from the Transformers library

For the tokenizer above, we used `AutoTokenizer` to load the right tokenizer, given a model name string. In a similar fashion, we use `AutoModel` here to load the right model. If you want to use another model, just change the name string in the hyperparameter settings below. (As mentioned above, see [this page](https://huggingface.co/transformers/pretrained_models.html) for a list of pre-trained models.) We'll typically get a warning message here, which we can ignore.

In [ ]:
from transformers import AutoModel, AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained(model_name)
example_bert_model = AutoModel.from_pretrained(model_name)

For illustration, we can also print the layers of the model.

In [ ]:
example_bert_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Computing next-word probabilities

Let's exemplify how we can compute the next-word (unnormalized log) probabilities.

We tokenize and integer-encode an example text and see how the model predicts that this sentence will continue.

In [ ]:
input_ids = tokenizer('I went to the bar to buy some', return_tensors='pt').input_ids

input_ids

tensor([[  40, 1816,  284,  262, 2318,  284, 2822,  617]])

We apply the model and it returns logits (unnormalized log probabilities). This tensor has the shape `(batch_size, n_words, voc_size)`.

In [ ]:
outputs = example_bert_model(input_ids).logits

We pick out the distribution for the last position and then find the index of the highest-scoring next word. This happens to be index 6099.

In [ ]:
most_probable_word_id = outputs[0, -1].argmax()

most_probable_word_id

tensor(6099)

... and index 6099 corresponds to the word "beer".

In [ ]:
tokenizer.decode(most_probable_word_id)

' beer'

Alternatively, we can *sample* from the distribution instead of finding the highest-scoring word. We can adjust the temperature to control the "randomness" of this sampling.

In [ ]:
from torch.distributions import Categorical

temperature = 0.5

dist = Categorical(logits=outputs[0, -1, :] / temperature)

sampled = dist.sample()

tokenizer.decode(sampled)

' drinks'

# Using the built-in generation function

However, in most applications we are not going to do the generation step-by-step as above, but simply use the built-in function `generate`.

If we simply call the `generate` method without any options, we will use *greedy* autoregressive decoding. For each step, we select the word (piece) that looks most likely, given the previous context.

The tensor `input_ids` corresponds to the *prompt* or prefix used to initialize the generation. By default, `generate` simply returns a tensor corresponding to the generated output. In our case, it will have the shape `(1, n)` where `n` is the length of the generated text. This tensor has one row because greedy decoding returns a single text.

Let's see what happens with a simple example! What is your impression? Do you think the GPT-2 model is close to [strong AI](https://en.wikipedia.org/wiki/Artificial_general_intelligence) or do we have a [*stochastic parrot*](https://dl.acm.org/doi/10.1145/3442188.3445922)? Any particular properties that you think are "odd" here?

In [ ]:
input_ids = tokenizer('I told him that', return_tensors='pt').input_ids

outputs = example_bert_model.generate(
    input_ids,
    max_length=50,
    pad_token_id=0
)

tokenizer.decode(outputs[0])

"I told him that I was going to be a part of the team. He said, 'I'm going to be a part of the team.' I said, 'I'm going to be a part of the team.' He said, 'I"

### Improving the generated output

**Beam search.** Greedy decoding will not necessarily find the most probable sequence according to the LM. *Beam search* can mitigate some of this problem. In beam search, we keep several alternative hypotheses and update each of them in each step.

To use beam search here, simply add the option `num_beams` and set it to an integer greater than 1. Since the beam search generates several texts, you may also add the option `num_return_sequences` and set it to something greater than 1 (but at most equal to the number of beams).

Does beam search seem to give you better generated texts than before?

**Avoiding repetition.** Probably, you previously encountered the problem that the decoder goes into an infinite loop and generates repetetive phrases. (Do you have an intuition for why this could be the case?) We may avoid this simply by forcing the model not to generate subsequences we have already seen. If you add the option `no_repeat_ngram_size` and set it to e.g. 2, then we prevent the model from generating previously seen bigrams (word pairs).

**Sampling.** Another way to improve the diversity of generated texts and to make them less bland is to introduce some randomness when generating. If set the option `do_sample` to `True`, we *sample* from the distribution at each step instead of selecting the most likely next word.

The `temperature` option (a floating-point value greater than 0) can be used to control the distribution used when sampling. When computing the probabilities over the vocabulary, the logits are divided by the `temperature` before applying the softmax. **Self-check:** What happens if `temperature` is close to 0 and what happens if it is greater than 1?

Other options that control the sampling distribution include `top_k` and `top_p`. You can read about them in [this notebook](https://github.com/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb) if you scroll down to the section *Top-K Sampling*.

We create an example input. This batch consists of a single sentence. We use the tokenizer above to encode the words as integers. (As above, special dummy tokens have been added at the beginning and end.)

In [ ]:
test_input = tokenizer(['The sun is shining .'], return_tensors='pt')
test_input.input_ids.shape

We can now apply the BERT model and get the output:

In [ ]:
bert_output = example_bert_model(input_ids=test_input.input_ids,
                                 attention_mask=test_input.attention_mask,
                                 output_attentions=True, output_hidden_states=True)

You can read about the output data structure here: https://huggingface.co/docs/transformers/main_classes/output

The parts of this data structure that are most interesting for us are:

- `last_hidden_state` (always returned)
- `hidden_states` (returned because we set `output_hidden_states` to `True`)
- `attentions` (returned because we set `attentions` to `True`)

Take a look at these objects and try to understand why they have the shapes they have. (Note: `hidden_states` and `attentions` are tuples.)

### Visualizing attention

The paper [*What does BERT look at? An Analysis of BERT's Attention*](https://aclanthology.org/W19-4828.pdf) by Clark et al (2019) describes an analysis of BERT's attention heads using visualization.

There is [an online tool](https://huggingface.co/exbert/?model=bert-base-uncased&modelKind=bidirectional) that can help you create visualizations similar to the paper.

You can also create visualizations in this notebook directly. The code below visualizes the attention scores using heatmaps.
**Note**: keep in mind that in the paper and in the visualization tool, the layers and attention heads are numbered starting from 1. So if you want to visualize the head 11-6 as in Figure 1 in the paper, you will have to write
```
bert_outputs.attentions[10][0, 5]
```

In [ ]:
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import numpy as np
import warnings

def plot_attention(input_ids, tokenizer, scores):
    scores = scores.detach().numpy()

    words = [tokenizer.decode(i) for i in input_ids]

    if tokenizer.pad_token in words:
        ix = words.index(tokenizer.pad_token)
        words = words[:ix]
        scores = scores[:ix, :ix]

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        fig, ax = plt.subplots()
        heatmap = ax.pcolor(scores, cmap='Blues_r')

        ax.set_xticklabels(words, minor=False, rotation='vertical')
        ax.set_yticklabels(words, minor=False)

        ax.xaxis.tick_top()
        ax.set_xticks(np.arange(scores.shape[1]) + 0.5, minor=False)
        ax.set_yticks(np.arange(scores.shape[0]) + 0.5, minor=False)
        ax.invert_yaxis()

        plt.colorbar(heatmap)

For instance, this code shows how head 1 in layer 3 attends to the next token as exemplified in Figure 1 in the paper.

In [ ]:
layer = 3
head = 1
sentence = 0
plot_attention(test_input.input_ids[sentence], tokenizer, bert_output.attentions[layer-1][sentence, head-1])

Try out the visualization tool or the heatmap visualization function in this notebook and try to see if you can see similar effects as described in the paper.

Do you think we can learn anything about a model by this kind of inspection? There have been debates in the research community, for instance in the following papers:
- Jain and Wallace (2019) [*Attention is not explanation*](https://aclanthology.org/N19-1357.pdf)
- Wiegreffe and Pinter (2019) [*Attention is not not explanation*](https://aclanthology.org/D19-1002.pdf)
- Bastings and Filippova (2020) [*The elephant in the interpretability room: Why use attention as explanation when we have saliency methods?*](https://aclanthology.org/2020.blackboxnlp-1.14.pdf)

### Masked language modeling

The BERT model was trained in a multitask fashion where one of the two training tasks is *masked language modeling*: for some tokens in the sentence, we substitute the dummy symbol `[MASK]` and then train the model to guess the missing words.

This masked language model can also be loaded via the library: we just need to use `AutoModelForMaskedLM` in this case. This loads the BERT model again, but also includes the "head" that predicts the missing tokens.

In [ ]:
from transformers import AutoModelForMaskedLM

mlm = AutoModelForMaskedLM.from_pretrained(model_name)

The output of applying `mlm` is a data structure where the relevant part is a tensor called `logits`. What is the shape of this tensor?

Can you see what the BERT model predicts for the masked token in the sentence below?

**Hint:** In this case, the masked token is at position 5 (because a `[CLS]` token is added at position 0). In general, you can write something like the following to find the position of the masked token:
```
mask_position = list(masked_sentence.input_ids[0]).index(tokenizer.mask_token_id)
```

**Hint:** You may found it useful to apply either [`argmax`](https://pytorch.org/docs/stable/generated/torch.argmax.html) or [`topk`](https://pytorch.org/docs/stable/generated/torch.topk.html).

**Hint:** If you know the index `i` in the vocabulary for some token, you can find the corresponding word string by calling `tokenizer.decode(i)`.

In [ ]:
masked_sentence = tokenizer('I went to the [MASK] to learn how to code.', return_tensors='pt')

In the paper by Petroni et al. (2019) called [*Language Models as Knowledge Bases?*](https://aclanthology.org/D19-1250.pdf), it is claimed that models such as BERT implicitly store quite a bit of encyclopedic knowledge that we can probe using the masked language model. Try to probe the model using sentences such as `[MASK] is the capital of Sweden.` and see if this idea seems reasonable. Can you think of any drawbacks of this approach?

Can you think of other experiments that would be interesting to carry out using the masked language model?

### Fine-tuning a BERT  model for document classification

For the classification task, we'll use our usual Amazon review benchmark. Some of the code here is identical to the previous notebook on document classifiction (with CBoW and BoW), with some small changes to adapt the code for the way that BERT deals with tokenization and vocabularies.

**Note**: the library includes ready-made solutions for document classification based on BERT (see [`BertForSequenceClassification`](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertForSequenceClassification)), and it may seem a bit redundant that we design our own classifier on top of BERT. But the point here is that we'd like to show how to use BERT in a modular way as a general representation component, so that you can use it in other types of applications. You may also read about the [`Trainer`](https://huggingface.co/docs/transformers/main_classes/trainer) functionality as an alternative to the plain PyTorch solution we are using here to deal with preprocessing, training loop etc.

Now, let's use a BERT model as we previously used other representation components (e.g. CBoW, RNN). There's only one small issue to keep in mind here: BERT gives *word* representations, but how do we get *document* representations? The recommended solution with BERT is to extract the representation at the dummy `[CLS]` token at position 0.

<img src="http://www.cse.chalmers.se/~richajo/nlp2019/l5/bert_class.png" alt="BERT" style="width:30%;"/>

Now, please implement the code to build a document classifier on top of a BERT model. You will first need to declare the classification head (output unit) in `__init__`, and then implement the computation in `forward`.

In [ ]:
from transformers import AutoModel
from torch import nn

class TextClassifierUsingBERT(nn.Module):

    def __init__(self, clf):
        super().__init__()

        print('Loading pre-trained model...')
        self.bert_model = AutoModel.from_pretrained(clf.params.bert_model_name)

        # Size of the pre-trained model's word representations.
        hidden_size = self.bert_model.config.hidden_size

        # Number of classes for the classification task.
        nbr_classes = clf.n_classes

        # Also don't forget to create a classification "head" (output unit).

    def forward(self, Xbatch, Xmask):

        # Xbatch is the document tensor of shape (n_docs, max_length), where
        # n_docs is the number of documents in the batch, and max_length is
        # the maximal document length in the batch. Since we truncate the
        # documents to length 128 in this example, max_length will be 128
        # in most cases.
        # Xmask is the attention mask: it is 0 for the positions corresponding to
        # the padding tokens, and 1 elsewhere.

        # Your code here...
        # As usual, print the shapes of the tensors if you are confused!

        # When you have implemented this, the output shape should be (n_docs, nbr_classes)
        # as in our previous classifiers. This output contains the logits for the classes:
        # you do not have to apply a softmax.
        return SOMETHING

### Running the text classifier

Now, let's build the complete text classification model and train it on the dataset.

In [ ]:
!wget http://www.cse.chalmers.se/~richajo/waspnlp2024/dredze_amazon_reviews.zip
!unzip dredze_amazon_reviews.zip

In [ ]:
import pandas as pd
amazon_corpus = pd.read_csv('dredze_amazon_reviews.tsv', sep='\t', header=None, names=['product', 'sentiment', 'text'])

**NOTE**: we use a lightly modified version of the classification code we used in previous lectures. The cell containing this code is available below and you need to exectute that cell before this one. (We put it below to improve readability of the notebook.)

In my experiments, I got accuracies around 0.88 when I truncate the documents to 128 tokens and use the distilled BERT model (a "compressed" BERT model that runs somewhat faster). You can get higher accuracies if you use longer documents or larger BERT models, but you should be careful in that case so that you don't run out of GPU memory.

This model is completely unregularized and the models seem to overfit quite quickly, with the accuracies sometimes peaking in the first epoch. Since each epoch takes about 2 minutes on my machine, so it's of course nice that we don't need to run it for many epochs.

In [ ]:
class TextClassifierParametersBERT:
    val_size = 0.2
    device = 'cuda'
    n_epochs = 4
    batch_size = 32
    decay = 0
    max_len = 128
    bert_model_name = 'distilbert-base-uncased'

    # The BERT paper recommends learning rates 5e-5, 3e-5, or 2e-5
    learning_rate = 5e-5

#X, Y = read_data('dredze_amazon_reviews.txt', use_sentiment=True)


torch.manual_seed(0)
clf = NNClassifier(TextClassifierParametersBERT(), TextClassifierUsingBERT)

clf.fit(list(amazon_corpus.text), list(amazon_corpus.sentiment))

If we want, we can run the classifier interactively on some test documents. How would you think a bag-of-words classifier would have handled these cases?

In [ ]:
clf.predict(['A bad song!', 'A great song!', 'Not a bad song!', 'Not a great song!'])

# Classification code

This is almost the same code as we used for previous classifiers. The *major* difference is that we do not use our own vocabulary, but instead rely on BERT's built-in tokenizer. Apart from this, there are only some small cosmetic changes.

Make sure you execute this cell before you run the classifier above.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer

import numpy as np
import time
import sys
from collections import defaultdict, Counter

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

class DocumentBatcher:
    """A collator that builds a batch from a number of documents."""

    def __init__(self, pad_id):
        self.pad_id = pad_id

    def make_batch_1(self, X):
        """Build a batch from a number of documents.
        Returns a tensor of shape [n_docs, max_doc_length]."""

        # How long is the longest document in this batch?
        max_len = max(len(x) for x in X)

        # Build the document tensor. We pad the shorter documents so that all documents
        # have the same length.
        Xpadded = torch.as_tensor([x + [self.pad_id]*(max_len-len(x)) for x in X])
        return Xpadded


    def make_batch_2(self, XY):
        """Build a batch from a number of documents and their labels.
        Returns two tensors X and Y, where X is the document tensor,
        of shape [n_docs, max_doc_length]

        and

        Y is the label tensor, of shape [n_docs].
        """

        # How long is the longest document in this batch?
        max_len = max(len(x) for x, _ in XY)

        # Build the document tensor. We pad the shorter documents so that all documents
        # have the same length.
        Xpadded = torch.as_tensor([x + [self.pad_id]*(max_len-len(x)) for x, _ in XY])

        # Build the label tensor.
        Y = torch.as_tensor([y for _, y in XY])

        return Xpadded, Y


    def __call__(self, instances):
        if isinstance(instances[0], tuple):
            return self.make_batch_2(instances)
        else:
            return self.make_batch_1(instances)


class NNClassifier:
    """A classifier based on a neural network."""

    def __init__(self, params, model_factory):
        self.params = params
        self.model_factory = model_factory
        print('Loading tokenizer...')
        self.tokenizer = AutoTokenizer.from_pretrained(params.bert_model_name)

    def preprocess(self, X, Y):

        Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size=self.params.val_size, random_state=0)

        self.lbl_enc = LabelEncoder()
        self.lbl_enc.fit(Ytrain)
        self.n_classes = len(self.lbl_enc.classes_)

        Xtrain_encoded = self.tokenizer(Xtrain, truncation=True, max_length=128).input_ids
        Xval_encoded = self.tokenizer(Xval, truncation=True, max_length=128).input_ids

        train_dataset = list(zip(Xtrain_encoded, self.lbl_enc.transform(Ytrain)))
        val_dataset = list(zip(Xval_encoded, self.lbl_enc.transform(Yval)))

        batcher = DocumentBatcher(self.tokenizer.pad_token_id)

        self.train_loader = DataLoader(train_dataset, self.params.batch_size, shuffle=True, collate_fn=batcher)
        self.val_loader = DataLoader(val_dataset, self.params.batch_size, shuffle=False, collate_fn=batcher)


    def fit(self, X, Y):
        """Train the model. We assume that a dataset and a model have already been provided."""
        par = self.params

        self.preprocess(X, Y)

        self.model = self.model_factory(self)
        self.model.to(par.device)

        self.loss_func = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=par.learning_rate, weight_decay=par.decay)

        self.history = defaultdict(list)

        for epoch in range(par.n_epochs):
            print(f'*** EPOCH {epoch+1} ***')

            t0 = time.time()

            self.model.train()
            train_loss, train_acc = self.epoch(self.train_loader, optimizer)

            self.model.eval()
            with torch.no_grad():
                val_loss, val_acc = self.epoch(self.val_loader)

            t1 = time.time()

            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['time'].append(t1-t0)

            print(f'Time: {t1-t0:.4f}; train loss: {train_loss:.4f}; train acc: {train_acc:.4f}; val loss: {val_loss:.4f}; val acc: {val_acc:.4f}')

    def epoch(self, batches, optimizer=None):
        """Runs the neural network for one epoch, using the given batches.
        If an optimizer is provided, this is training data and we will update the model
        after each batch. Otherwise, this is assumed to be validation data.

        Returns the loss and accuracy over the epoch."""
        n_correct = 0
        n_instances = 0
        total_loss = 0

        for step, (Xbatch, Ybatch) in enumerate(batches, 1):
            Xbatch = Xbatch.to(self.params.device)
            Xmask = (Xbatch != self.tokenizer.pad_token_id).long()
            Ybatch = Ybatch.to(self.params.device)

            scores = self.model(Xbatch, Xmask)

            loss = self.loss_func(scores, Ybatch)
            total_loss += loss.item()

            guesses = scores.argmax(dim=1)
            n_correct += (guesses == Ybatch).sum().item()
            n_instances += Ybatch.shape[0]

            if optimizer:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            print('.', end='')
            if step % 50 == 0:
              print(f' ({step})')

            sys.stdout.flush()

        print()

        return total_loss/len(batches), n_correct/n_instances


    def predict(self, X):
        """Run a trained classifier on a set of instances and return the predictions."""

        batcher = DocumentBatcher(self.tokenizer.pad_token_id)
        encoded = self.tokenizer(X).input_ids
        loader = DataLoader(encoded, self.params.batch_size, collate_fn=batcher)

        self.model.eval()
        outputs = []
        with torch.no_grad():
            for Xbatch in loader:
                Xbatch = Xbatch.to(self.params.device)
                Xmask = (Xbatch != self.tokenizer.pad_token_id).long()
                scores = self.model(Xbatch, Xmask)
                guesses = scores.argmax(dim=1)
                outputs.append(guesses.cpu().numpy())

        return self.lbl_enc.inverse_transform(np.hstack(outputs))


In [ ]:
!wget https://www.cse.chalmers.se/~richajo/diverse/l7/books.data
!wget https://www.cse.chalmers.se/~richajo/diverse/l7/s7_pretrained.model

In [ ]:
with open('books.data', 'rb') as f:
    books_X, books_Y = pickle.load(f)

test_input = torch.as_tensor(books_X)
print('X shape:', books_X.shape)
print('Y length:', len(books_Y))

split_ix = 1500
books_X_tr = books_X[:split_ix]
books_Y_tr = books_Y[:split_ix]
books_X_te = books_X[split_ix:]
books_Y_te = books_Y[split_ix:]

In [ ]:
model_fp32 = torch.load('s7_pretrained.model')
model_bf16 = torch.load('s7_pretrained.model')
model_bf16.bfloat16();

In [ ]:
def batcher(batch):
    X = torch.as_tensor([x for x, _ in batch])
    Y = 1.0*torch.as_tensor([y for _, y in batch])
    return X, Y

In [ ]:
def eval_model(model):
    dl = DataLoader(list(zip(books_X_te, books_Y_te)), batch_size=32, shuffle=False, collate_fn=batcher)
    n_corr = 0
    for Xb, Yb in dl:
        with torch.no_grad():
            model_out = model(Xb)
        preds = model_out[:, 0] > 0
        gold = Yb > 0
        n_corr += sum(preds == gold).item()
    return n_corr / len(books_Y_te)

In [ ]:
eval_model(model_fp32)

# Basic fine-tuning

We create a new model where we copy the weights from the pre-trained model.

In [ ]:
torch.manual_seed(0)

finetuned = nn.Sequential(
    nn.Linear(in_features=768, out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=1)
)

# pretrained = torch.load('s7_pretrained.model')

finetuned[0].weight.data = pretrained[0].weight.data.clone()
finetuned[0].bias.data = pretrained[0].bias.data.clone()
finetuned[2].weight.data = pretrained[2].weight.data.clone()
finetuned[2].bias.data = pretrained[2].bias.data.clone()

In [ ]:
eval_model(finetuned)

# Basic fine-tuning

We create a new model where we copy the weights from the pre-trained model.

In [ ]:
out_fp32 = model_fp32(test_input)

- How much do the output values differ between FP32 and BF16?
- How many binary predictions are "flipped" when we move to lower precision?

In [ ]:
import torch.quantization
quantized_model = torch.quantization.quantize_dynamic(model_fp32, {torch.nn.Linear}, dtype=torch.qint8)

- How much do the output values differ between FP32 and 8-bit integer quantization?
- How many binary predictions are "flipped" when we use 8-bit integer quantization?

In [ ]:
def train(model, n_epochs=10):
    dl = DataLoader(list(zip(books_X_tr, books_Y_tr)), batch_size=32, shuffle=True, collate_fn=batcher)

    # NOTE!
    params = [ p for p in model.parameters() if p.requires_grad_ ]

    optimizer = torch.optim.Adam(params, lr=1e-3)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(n_epochs):
        total_loss = 0
        for Xb, Yb in dl:
            model_out = model(Xb)[:, 0]
            loss = loss_fn(model_out, Yb)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        mean_loss = total_loss / len(dl)
        acc = eval_model(model)
        print(f'loss = {mean_loss:.4f}, acc = {acc:.4f}')

Your task:
- Complete `count_trainable_parameters` below.
- Count the total number of trainable parameters in the model you fine-tuned.
- Use the function `train` to fine-tune the cloned model.

In [ ]:
def count_trainable_parameters(model):
  # TODO
  return None

# Implementing LoRA

In [ ]:
class LinearBlockWithLoRA(nn.Module):

    def __init__(self, W, r):
        super().__init__()
        self.W = W
        in_dimension, out_dimension = W.weight.shape

        self.A = nn.Linear(in_features=in_dimension, out_features=r, bias=False)
        self.B = nn.Linear(in_features=r, out_features=out_dimension, bias=False)

    def forward(self, X):
        W_out = self.W(X)

        a_out = self.A(X)
        b_out = self.B(a_out)

        # TODO: add more code here
        return W_out + b_out

Your task:
- Complete `LinearBlockWithLoRA` above
- Set up a model using this new block to replace the first linear layer. Initialize parameters from the pre-trained model. (Don't forget to switch off gradient computation for `W`.)
- Count the parameters in the new model.
- Train the new model.